# Descriptive Statistics for Common Datasets
This script can be run on each of the datasets. Dataset-specific scripts will be written separately.
This script counts and removes duplicate rows, calculates the number of unique values per column, calculates the number/percent of nulls per column, and counts number of rows with the same set of null column(s).

_Author: Jared Gauntt_

## Prepare for Analysis


### Set Parameters

In [1]:
localFolder='../data/01_raw/' #If run from Git Hub project folder structure, script must be in "notebooks" folder
fileName='20210225-ems-raw-v04.xlsx'
sheetName='Patients'
removeDuplicates=True

### Import Libraries

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Import From Excel Spreadsheet

In [3]:
#Import single tab 
df=pd.read_excel(localFolder+fileName,sheet_name=sheetName)

#Print (1) number of rows/columns and (2) column names/types for quick confirmation of successful import
numOriginalRows=len(df)
numCols=len(df.columns)
print(sheetName)
print('Number of Rows = '+str(numOriginalRows))
print('Number of Columns = '+str(numCols))
df.dtypes

Patients
Number of Rows = 543774
Number of Columns = 12


PatientId                         int64
FRDPersonnelID                   object
Shift                            object
UnitId                           object
FireStation                       int64
Battalion                         int64
PatientOutcome                   object
PatientGender                    object
CrewMemberRoles                  object
DispatchTime             datetime64[ns]
FRDPersonnelGender               object
FRDPersonnelStartDate    datetime64[ns]
dtype: object

## Numeric Analysis

### Duplicate Rows
Duplicate rows will likely need to be removed during data conditioning.

In [4]:
#Determine which rows are duplicates (True=duplicate, False=first instance of row)
duplicateRowIdentifier=df.duplicated()

#Calculate number of duplicate rows
numDuplicateRows=len(duplicateRowIdentifier[duplicateRowIdentifier==True])
numUniqueRows=numOriginalRows-numDuplicateRows

#Calculate percentage of rows that are duplicates
percentDuplicateRows=round(numDuplicateRows/len(df)*100,4)

#Print results
print('Number of Duplicate Rows =')
print(numDuplicateRows)
print('Percentage of Rows that are Duplicates = ')
print(percentDuplicateRows)

Number of Duplicate Rows =
14
Percentage of Rows that are Duplicates = 
0.0026


Duplicate rows will be removed before conducting the remaining analysis in this script.

In [5]:
if removeDuplicates:
    #Reduce to the rows that were not flagged as duplicates
    df=df.loc[duplicateRowIdentifier==False,:]
    print('Duplicates Removed')
    print('Expected Number of Rows = '+str(numUniqueRows))
    print('Updated Data Frame Shape =')
    print(df.shape)
else:
    print('Duplicate rows NOT Removed')

Duplicates Removed
Expected Number of Rows = 543760
Updated Data Frame Shape =
(543760, 12)


### Unique Values Per Column
This is a simple calculation of the number of unique values per individual column.

In [6]:
#Calculate the number of unique values per columns (NULL counts as a value)
dsNumUnique=df.nunique(dropna=False)
dsNumUnique.name='Number of Unique Values'
print(dsNumUnique)

PatientId                246929
FRDPersonnelID             1833
Shift                         3
UnitId                      167
FireStation                  38
Battalion                     7
PatientOutcome               13
PatientGender                 4
CrewMemberRoles             204
DispatchTime             244490
FRDPersonnelGender            3
FRDPersonnelStartDate       459
Name: Number of Unique Values, dtype: int64


### Null Values Per Column
This is a simple calculation of the number/percentage of null values per individual column.

In [7]:
#Calculate the number of null values per columns
dsNumNull=df.isnull().sum()
dsNumNull.name='Number Rows With Nulls'
print(dsNumNull)

PatientId                     0
FRDPersonnelID                0
Shift                         0
UnitId                        0
FireStation                   0
Battalion                     0
PatientOutcome                2
PatientGender            131378
CrewMemberRoles               0
DispatchTime                  0
FRDPersonnelGender          462
FRDPersonnelStartDate       462
Name: Number Rows With Nulls, dtype: int64


In [8]:
#Calculate the percentage of null values per columns
dsPercentNull=(dsNumNull/len(df)*100).round(2)
dsPercentNull.name='Percent Rows With Nulls'
print(dsPercentNull)

PatientId                 0.00
FRDPersonnelID            0.00
Shift                     0.00
UnitId                    0.00
FireStation               0.00
Battalion                 0.00
PatientOutcome            0.00
PatientGender            24.16
CrewMemberRoles           0.00
DispatchTime              0.00
FRDPersonnelGender        0.08
FRDPersonnelStartDate     0.08
Name: Percent Rows With Nulls, dtype: float64


In [9]:
#Merge data series (by column) together
dfPerCol=pd.DataFrame()
dfPerCol[dsNumUnique.name]=dsNumUnique
dfPerCol[dsNumNull.name]=dsNumNull
dfPerCol[dsPercentNull.name]=dsPercentNull

### Null Values Per Row
This section determines which columns have a null value per each row, resulting in a data series matching the number of unique rows in the dataset. The series values are tuples of column names. The resulting table shows the unique tuples of column names with their corresponding row count. Once the data subsets for the project questions are determined, this will help inform the completeness of those data subsets.

In [10]:
#For a single row, determine which columns have null values
def NullsPerRow(dsRow):  
    columnsNull=list(dsRow[dsRow.isnull()].index)
    columnsNull.sort()
    columnsNull=tuple(columnsNull) #used tuple since lists can't be used by pandas unique()
    return(columnsNull)
dsNulls=df.apply(NullsPerRow,axis=1)

#Create data frame for counting 
dfNulls=pd.DataFrame(dsNulls.unique(),columns=['Columns With Null'])
dfNulls['Number of Rows']=0

#Count the number of rows per each tuple of null columns
for index in dfNulls.index:
    dfNulls.loc[index,'Number of Rows']=len(dsNulls[dsNulls==dfNulls.loc[index,'Columns With Null']])
dfNulls.sort_values(by='Number of Rows',ascending=False,inplace=True)
dfNulls.reset_index(drop=True,inplace=True)

#Add a column for percent of rows
dfNulls['Percent of Rows']=(dfNulls['Number of Rows']/dfNulls['Number of Rows'].sum()*100).round(2)
dfNulls

,Columns With Null,Number of Rows,Percent of Rows
0,(),412008,75.77
1,"(PatientGender,)",131288,24.14
2,"(FRDPersonnelGender, FRDPersonnelStartDate)",374,0.07
3,"(FRDPersonnelGender, FRDPersonnelStartDate, PatientGender)",88,0.02
4,"(PatientGender, PatientOutcome)",2,0.00


## Visualizations

In [11]:
print(sheetName+' Dataset\n')
print('Original Number of Rows = '+str(numOriginalRows))
print('Duplicate Number of Rows = '+str(numDuplicateRows))
print('Percent Duplicate Rows = '+str(percentDuplicateRows)+'\n')
if removeDuplicates:
    print('Duplicate rows removed prior to remaining analysis')
    print('Unique Number of Rows = '+str(numUniqueRows)+'\n')
else:
    print('Duplicate Rows NOT Removed')

Patients Dataset

Original Number of Rows = 543774
Duplicate Number of Rows = 14
Percent Duplicate Rows = 0.0026

Duplicate rows removed prior to remaining analysis
Unique Number of Rows = 543760



In [12]:
dfPerCol

,Number of Unique Values,Number Rows With Nulls,Percent Rows With Nulls
PatientId,246929,0,0.00
FRDPersonnelID,1833,0,0.00
Shift,3,0,0.00
UnitId,167,0,0.00
FireStation,38,0,0.00
Battalion,7,0,0.00
PatientOutcome,13,2,0.00
PatientGender,4,131378,24.16
CrewMemberRoles,204,0,0.00
DispatchTime,244490,0,0.00


"Columns With Null" shows that for a single row, these columns had null values. If we find that one of our data subsets (for answering specific questions) has the same columns as one of these tuples with a high count, there might be a problem with too many nulls.

In [13]:
dfNulls

,Columns With Null,Number of Rows,Percent of Rows
0,(),412008,75.77
1,"(PatientGender,)",131288,24.14
2,"(FRDPersonnelGender, FRDPersonnelStartDate)",374,0.07
3,"(FRDPersonnelGender, FRDPersonnelStartDate, PatientGender)",88,0.02
4,"(PatientGender, PatientOutcome)",2,0.00
